In [3]:
import minimalmodbus
import serial

instrument = minimalmodbus.Instrument('COM12', 1) # port name, slave address (in decimal)
instrument.serial.baudrate = 9600   # Baud
instrument.serial.parity   = serial.PARITY_ODD

#modbus seems to have an unreliable connection, so write a wrapper for the read and write functions that loops until the read or write is successful, handle the exception if the read or write is not successful
def read_register(address, number_of_decimals):
    while True:
        try:
            return instrument.read_register(address, number_of_decimals)
        except:
            pass

def write_register(address, value, number_of_decimals):
    while True:
        try:
            instrument.write_register(address, value, number_of_decimals)
            break
        except:
            pass
        
def setTemperature(temperature):
    write_register(0x24, temperature, 1)

def getTemperatureSP():
    return read_register(0x24, 1)

def getTemperature():
    return read_register(0x23, 1)

def setHumidity(humidity):
    write_register(0x29, humidity, 1)

def getHumiditySP():
    return read_register(0x29, 1)

def getHumidity():
    return read_register(0x28, 1)

def enableManualOverride():
    write_register(0x09, 0x30, 0)

def startProgram():
    write_register(0x0E, 0x01, 0)

In [4]:
from time import sleep

#enable manual override by writing to register 0x0009
enableManualOverride()
sleep(1)

#set the temp to 10 degrees by writing to register 0x0024
setTemperature(12.00)
sleep(1)
confirm = getTemperatureSP()
print('confirm', confirm)
temp = getTemperature()
print('temp', temp)

#set the humidity to 0% by writing to register 0x0029
setHumidity(0)
sleep(1)
confirm = getHumiditySP()
print('confirm', confirm)
humidity = getHumidity()
print('humidity', humidity)

sleep(1)

#start the program by writing to register 0x000E
startProgram()

confirm 12
temp 21.1
confirm 0
humidity 0


In [ ]:
#loop until the temperature is at the current SP, and then adjust the temperature to 15 degrees. loop every 5 seconds
tempSP = getTemperatureSP()
while True:
    temp = getTemperature()
    if temp == tempSP:
        break
    sleep(5)

#adjust the temperature to 15 degrees
setTemperature(15)